In [1]:
import pandas as pd # used for data import
import numpy as np # used for numerical operations
import torch # used for tensor operations
import torch.nn as nn # used for building neural networks
from torch.utils.data import DataLoader, TensorDataset # used for creating data loaders
from sklearn.preprocessing import StandardScaler # used for standardizing features
from sklearn.metrics import precision_recall_curve, average_precision_score # used for evaluating models
import statsmodels.api as sm # used for probit & logit regression
import matplotlib.pyplot as plt # used for plotting PR curves

np.random.seed(42) # set random seed for reproducibility
torch.manual_seed(42)

In [2]:
train_df = pd.read_csv('../全连接神经网络/train_data.csv') # import training data
test_df = pd.read_csv('../全连接神经网络/test_data.csv')
train_df.head(5) # display first 5 observations of training data

# transform DataFrame into numpy ndarray
X_train_array = train_df.drop(columns=['noncompliance']).values
y_train_array = train_df['noncompliance'].values
X_test_array = test_df.drop(columns=['noncompliance']).values
y_test_array = test_df['noncompliance'].values

# standardize input features
scaler = StandardScaler() # initialize the scaler
X_train_array = scaler.fit_transform(X_train_array)
X_test_array = scaler.transform(X_test_array) # standardize test data using the train scaler

# transform numpy ndarray into torch.tensor
X_train_tensor = torch.tensor(X_train_array, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_array, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_array, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_array, dtype=torch.float32)

# bundle feature and label into TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# create DataLoader
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True) # shuffle means randomize the order of observations
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [3]:
class FNN(nn.Module):
    """
    A fully connected neural network for binary classification.
    """
    def __init__(self):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(24, 64) # input layer: 25 features -> 64 neurons
        self.fc2 = nn.Linear(64, 32) # hidden layer1: 64 neurons -> 32 neurons
        self.fc3 = nn.Linear(32, 8) # hidden layer2: 32 neurons -> 8 neurons
        self.fc4 = nn.Linear(8, 1) # hidden layer3: 8 neurons -> output probability
        
    def forward(self, x):
        x = torch.relu(self.fc1(x)) # relu activation
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x)) # sigmoid activation
        return x
    
def train(model, train_loader, criterion, optimizer, num_epochs):
    """
    functions used to train the model
    params:
        model: the neural network to be trained;
        train_loader: a DataLoader object, trainging dataset;
        criterion: loss function;
        optimizer: optimiser that updates parameters after backpropagation;
        num_epochs: number of epochs in optimization;
    return:
        None
    """
    model.train() # set the model to training mode
    for epoch in range(num_epochs): # loop over epochs
        for batch_idx, (data, target) in enumerate(train_loader): # loop over batches
            optimizer.zero_grad() # clear previous gradients
            output = model(data) # do forward propagation
            loss = criterion(output.squeeze(1), target) # calculate loss
            loss.backward() # do backpropagation
            optimizer.step() # update parameters

            if (epoch + 1) % 50 == 0 and batch_idx == len(train_loader)-2: # print second last batch's loss every 50 epochs
                print(f"Epoch [{epoch+1}/{num_epochs}] Batch {batch_idx+1} Loss: {loss.item():.4f}")

def test(model, test_loader, criterion):
    """
    Run the trained model on the test set.
    params:
        model: the network after trained;
        test_loader: DataLoader, test dataset;
        criterion: loss function;
    return:
        pred_probs: tensor, predicted test probabilities;
    """
    model.eval() # set the model to evaluation mode
    test_loss = 0.0
    pred_probs = [] # a list used to accumulated each batch's predicted probabilities
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data).squeeze(1) # forward propagation using test data and return output probabilities
            test_loss += criterion(output, target).item() # accumulate loss
            pred_probs.append(output) # accumulate predicted probabilityes
            
    test_loss /= len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.4f}")
    
    return torch.cat(pred_probs)

In [6]:
model = FNN() # instantiate the model
criterion = nn.BCELoss() # binary cross entropy loss
optimizer_adam = torch.optim.Adam(model.parameters(), lr=3e-4)
train(model, train_loader, criterion, optimizer_adam, num_epochs=500)


Epoch [50/500] Batch 193 Loss: 0.1711
Epoch [100/500] Batch 193 Loss: 0.0876
Epoch [150/500] Batch 193 Loss: 0.0569
Epoch [200/500] Batch 193 Loss: 0.1984
Epoch [250/500] Batch 193 Loss: 0.1597
Epoch [300/500] Batch 193 Loss: 0.0789
Epoch [350/500] Batch 193 Loss: 0.0718
Epoch [400/500] Batch 193 Loss: 0.1066
Epoch [450/500] Batch 193 Loss: 0.1127
Epoch [500/500] Batch 193 Loss: 0.1025
Test Loss: 0.0022


tensor([0.0025, 0.0002, 0.0036,  ..., 0.0020, 0.0198, 0.0034])

In [5]:
def l2_norm_all_weights(model):
    total_norm = 0.0
    for name, param in model.named_parameters():
        if 'weight' in name and param.requires_grad:
            total_norm += torch.norm(param, p=2) ** 2
    return total_norm.sqrt()

l2_norm_all_weights(model) # check the l2 norm of all weights

tensor(19.3912, grad_fn=<SqrtBackward0>)

In [ ]:
test(model, test_loader, criterion)